In [86]:
from selenium.webdriver import Chrome, ChromeOptions
from bs4 import BeautifulSoup
import gzip
from pathlib import Path
from datetime import datetime
import re
from glob import glob
import pandas as pd

In [13]:
url = 'https://dunksandthrees.com/'
chromedriver_path = Path().joinpath(
    '/Users','johnschroeder','Documents','gambling_stuff','chromedriver')
base_path = '/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_team_ratings'

In [3]:
chrome_options = ChromeOptions()
chrome_options.add_argument('--headless')

In [4]:
driver = Chrome(executable_path=chromedriver_path, options=chrome_options)

In [5]:
response = driver.get(url)

In [6]:
soup = BeautifulSoup(driver.page_source)

In [14]:
raw_page_date = soup.find('span', {'class': 'page-date'}).text

In [19]:
ymd = str(datetime.strptime(raw_page_date, '%B %d, %Y').date()).split('-')

In [23]:
raw_file = Path(base_path).joinpath(*ymd, 'dunksandthrees.html')

In [9]:
team_stats = soup.find('div', {'id': 'team-stats'})

In [ ]:
base

In [26]:
Path(raw_file).parent.mkdir(parents=True, exist_ok=True)
with open(raw_file, 'w') as fp:
    fp.write(str(team_stats.prettify()))

In [28]:
# collect all raw files and combine to a single dataframe

In [29]:
curated_file = '/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv'

In [31]:
stats_headers = team_stats.find('tr', {'class': 'main-header-tr'})

In [39]:
df_headers = [re.sub('\W+', '', _.text).lower() for _ in stats_headers.find_all('th')]

In [40]:
df_headers

['team',
 'wl',
 'anet',
 'aortg',
 'adrtg',
 'sos',
 'osos',
 'dsos',
 'pace',
 'opl',
 'dpl']

In [ ]:
#team-stats-table > tbody > tr.stat-row.expanded

In [50]:
team_stats.find_all('tr', {'class': 'stat-row'})[0]

<tr class="stat-row entered" style="opacity: 1;"><td class="team-td" style="padding: 0px 0.2em;"><span style="vertical-align: middle;"><span class="small-span">1</span> UTA </span><img alt="" src="images/teams/UTA.png" style="vertical-align: middle;" width="20px"/></td><td class="wl-td">50-18</td><td class="stat-td adj_netrtg"><span class="val-span">+9.0</span><span class="rank-span" style="background-color: rgb(28, 118, 60);">1</span></td><td class="stat-td adj_ortg"><span class="val-span">117.1</span><span class="rank-span" style="background-color: rgb(29, 87, 51);">5</span></td><td class="stat-td adj_drtg right-border"><span class="val-span">108.0</span><span class="rank-span" style="background-color: rgb(28, 110, 58);">3</span></td><td class="stat-td sos"><span class="val-span">-0.1</span><span class="rank-span" style="background-color: rgb(29, 32, 37);">13</span></td><td class="stat-td sos_o"><span class="val-span">-0.1</span><span class="rank-span" style="background-color: rgb(25

In [58]:
team_rows = team_stats.find_all('tr', {'class': 'stat-row'})

In [59]:
team_rows[0]

<tr class="stat-row entered" style="opacity: 1;"><td class="team-td" style="padding: 0px 0.2em;"><span style="vertical-align: middle;"><span class="small-span">1</span> UTA </span><img alt="" src="images/teams/UTA.png" style="vertical-align: middle;" width="20px"/></td><td class="wl-td">50-18</td><td class="stat-td adj_netrtg"><span class="val-span">+9.0</span><span class="rank-span" style="background-color: rgb(28, 118, 60);">1</span></td><td class="stat-td adj_ortg"><span class="val-span">117.1</span><span class="rank-span" style="background-color: rgb(29, 87, 51);">5</span></td><td class="stat-td adj_drtg right-border"><span class="val-span">108.0</span><span class="rank-span" style="background-color: rgb(28, 110, 58);">3</span></td><td class="stat-td sos"><span class="val-span">-0.1</span><span class="rank-span" style="background-color: rgb(29, 32, 37);">13</span></td><td class="stat-td sos_o"><span class="val-span">-0.1</span><span class="rank-span" style="background-color: rgb(25

In [83]:
replaces = re.compile(r'(\+)|(\d+\s+)')
df_data = []
for team_row in team_rows:
    row_data = [re.sub(replaces, '', _.text).strip() for _ in team_row.find_all('td')]
    df_data.append(row_data)

In [96]:
incr_df = pd.DataFrame(df_data, columns=df_headers)
incr_df['date'] = str(datetime.strptime(raw_page_date, '%B %d, %Y').date())

In [91]:
current_curated_df = pd.read_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv')

In [92]:
current_curated_df.head()

,team,wl,anet,aortg,adrtg,sos,osos,dsos,pace,opl,dpl,date
0,UTA,50-18,9.0,117.1,108.0,-0.1,-0.0,-0.0,99.1,14.8,14.3,2021-05-08
1,LAC,45-22,6.2,117.3,111.0,-0.1,-0.1,-0.0,97.4,15.0,14.5,2021-05-08
2,PHI,47-21,5.6,113.1,107.5,-0.0,-0.0,-0.0,99.8,14.7,14.1,2021-05-08
3,MIL,43-24,5.5,116.7,111.2,-0.3,-0.0,-0.3,102.3,13.6,14.6,2021-05-08
4,DEN,44-24,5.0,117.1,112.1,-0.0,0.2,-0.2,97.3,15.1,14.5,2021-05-08


In [97]:
incr_df.head()

,team,wl,anet,aortg,adrtg,sos,osos,dsos,pace,opl,dpl,date
0,UTA,50-18,9.01,117.15,108.03,-0.113,-0.122,-0.014,99.115,14.818,14.37,2021-05-09
1,LAC,45-23,6.12,117.14,111.09,-0.114,-0.020,-0.017,97.427,15.026,14.618,2021-05-09
2,PHI,47-21,5.63,113.113,107.52,-0.012,-0.019,-0.016,99.812,14.716,14.11,2021-05-09
3,PHX,48-20,5.64,116.57,111.08,-0.122,0.18,-0.227,97.426,14.822,14.727,2021-05-09
4,MIL,43-24,5.55,116.76,111.210,-0.330,-0.123,-0.328,102.33,13.62,14.619,2021-05-09


In [100]:
revised_df = pd.concat([current_curated_df, incr_df]).sort_values('date', ascending=False)

In [102]:
revised_df.to_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_team_ratings/dunksandthrees.csv', index=False)